## Setup

In [1]:
# Get raw advent-of-code data
from aocd.models import Puzzle

puzzle = Puzzle(year=2025, day=6)
input_data = puzzle.input_data
example = puzzle.examples[0]

In [2]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

from common.utils.perf_check import check_time

## Part a

### Iterative approach
Let's see how far we get with  python builtins first.

In [3]:
# Imports
from math import prod

In [4]:
# Constants
op_str_to_func = {
    "*": prod,
    "+": sum,
}

In [5]:
# Functions
def solve_all_problems_iterative(input_data: str) -> int:
    """Solve all problems iteratively."""
    lines = input_data.splitlines()
    number_lists: list[tuple[int, ...]] = list(zip(*(map(int, line.split()) for line in lines[:-1]), strict=True))
    ops = lines[-1].split()

    total_result = 0
    for numbers, op in zip(number_lists, ops, strict=True):
        total_result += op_str_to_func[op](numbers)
    return total_result

In [100]:
# Correctness check
str(solve_all_problems_iterative(example.input_data)) == example.answer_a

True

In [101]:
# Performance check
iterative_time_a = check_time(solve_all_problems_iterative, input_data)
print(f"The iterative implementation takes {iterative_time_a:.1f} ms per run.")

The iterative implementation takes 0.7 ms per run.


### Vectorized approach
Let's see if we can speed things up with NumPy.

In [6]:
# Imports
import numpy as np

In [7]:
# Functions
def solve_all_problems_vectorized(input_data: str) -> int:
    """Solve all problems vectorized."""
    # Parse input data into numpy arrays
    lines = input_data.splitlines()
    numbers = np.loadtxt(lines[:-1], dtype=np.int64)
    ops = np.array(object=lines[-1].split())

    # Multiply the numbers where the operation is "*", sum the others
    # NOTE: this assumes that all operations are either "*" or "+"
    return int(np.where(ops == "*", np.prod(numbers, axis=0), np.sum(numbers, axis=0)).sum())

In [104]:
# Correctness check
str(solve_all_problems_vectorized(example.input_data)) == example.answer_a

True

In [105]:
# Performance check
vectorized_time_a = check_time(solve_all_problems_vectorized, input_data)
print(
    f"The vectorized implementation takes {vectorized_time_a:.2f} ms per run."
    f"\nThis is {iterative_time_a / vectorized_time_a:.1f}x faster than the iterative version."
)

The vectorized implementation takes 0.25 ms per run.
This is 2.7x faster than the iterative version.


In [106]:
# Submit answer
puzzle.answer_a = solve_all_problems_vectorized(input_data)

## Part b

### Iterative approach
Let's start with a pure-python, iterative approach again.

In [8]:
# Functions
def solve_all_cephalopod_problems_iterative(input_data: str) -> int:
    """Solve all problems written in the cephalopod language iteratively."""
    lines = input_data.splitlines()
    n_rows = len(lines) - 1

    # Transposing the input strings to get columns of number strings
    numbers_strs = ["".join(col) for col in zip(*lines[:-1], strict=True)][::-1]

    # Fill the operations list with the first operation
    ops = []
    number_lists = []
    prev_op_idx = 0

    # Parse operations
    for idx, op_char in enumerate(lines[-1][::-1], start=1):
        if op_char in "+*":
            # Found an operation, add it to the list
            ops.append(op_char)

            # Append the corresponding number list, removing any all-space strings
            number_lists.append(list(map(int, set(numbers_strs[prev_op_idx:idx]) - {" " * n_rows})))

            # Update previous operation index
            prev_op_idx = idx

    # The rest is similar to part a
    total_result = 0
    for numbers, op in zip(number_lists, ops, strict=True):
        total_result += op_str_to_func[op](numbers)
    return total_result

In [9]:
# Correctness check
str(solve_all_cephalopod_problems_iterative(example.input_data)) == example.answer_b

True

In [10]:
# Performance check
iterative_time_b = check_time(solve_all_cephalopod_problems_iterative, input_data)
print(f"The iterative implementation takes {iterative_time_b:.2f} ms per run.")

The iterative implementation takes 1.12 ms per run.


### Vectorized approach
Trying to vectorize with NumPy again. However, it was really difficult to make it faster than pure-python in this case, probably due to the fact that the problem arguments are of irregular length, and numpy arrays are not optimized for string operations.

In [ ]:
# Functions
def solve_all_cephalopod_problems_vectorized(input_data: str) -> int:
    """Solve all problems written in cephalopod language vectorized."""
    lines = input_data.splitlines()

    # Get transposed character matrix
    char_matrix = np.array(list(zip(*lines[:-1], strict=True)))
    n_rows = char_matrix.shape[1]

    # Get vertical number strings by joining columns and reversing order
    number_strs = np.array(["".join(col) for col in char_matrix.tolist()][::-1])

    # Find the separation indices within the number strings (these consist of spaces for all rows)
    number_col_is_sep = number_strs == " " * n_rows

    # Cast number strings to integers, replacing separators with 0
    numbers = np.where(number_col_is_sep, "0", number_strs).astype(np.int64)

    # Construct group separation indices
    group_ends = np.concatenate((np.flatnonzero(number_col_is_sep), np.array([len(number_strs)])))
    group_starts = np.concatenate(([0], group_ends[:-1] + 1))

    # Parse operations
    ops = np.array(lines[-1].split(), dtype="U1")[::-1]

    return sum(
        [  # Calculate total for each group based on operation
            int(np.prod(numbers[s:e])) if op == "*" else int(np.sum(numbers[s:e]))
            for s, e, op in zip(group_starts, group_ends, ops, strict=True)
        ]
    )

In [31]:
# Correctness check
str(solve_all_cephalopod_problems_vectorized(example.input_data)) == example.answer_b

True

In [36]:
# Performance check
vectorized_time_b = check_time(solve_all_cephalopod_problems_vectorized, input_data)

print(f"The vectorized implementation takes {vectorized_time_b:.3f} ms per run.")
print(f"This is {iterative_time_b / vectorized_time_b:.1f}x faster than the iterative implementation.")

The vectorized implementation takes 4.954 ms per run.
This is 0.2x faster than the iterative implementation.


In [34]:
# Submit answer
puzzle.answer_b = solve_all_cephalopod_problems_vectorized(input_data)

coerced int64 value np.int64(11159825706149) to '11159825706149'


That's the right answer!  You are one gold star closer to decorating the North Pole.You have completed Day 6! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
